## Stack overflow developer survey 2019 -- analysis

### Business Understanding:

Stack overflow survey is one of the largest developer survey. Almost every developer has used Stack overflow atleast once while coding. Below are a few questions that I'd like to answer using the survey data.

1. Does open source development help you increase your income?
2. Does your undergrad major affect youe income?
3. What are the languages of choice for 2020?
4. Does Location, Open source contribution and dependents influence income?
5. Does years of coding experience influence salary?
6. What are the desired platforms for 2020?
7. What are the desired databases for 2020?
8. Which was the most used eidtor of 2019?

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
import seaborn as sns

%matplotlib inline

In [81]:
import warnings
warnings.filterwarnings('ignore')

### Data Understanding

Let us look at the data and try to assert whether the business questions listed can be answered or not.

In [82]:
survey_data = pd.read_csv('survey_results_public.csv')
survey_data.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [83]:
survey_data.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

Looking at the columns. We can conclude that the business questions listed can be answered.

### Question 1: Does open source development help you increase your income?

In [84]:
ques1_data = survey_data[['OpenSourcer', 'ConvertedComp']]
# dropping all rows without a ConvertedComp as this the column that I am trying to predict
ques1_data = ques1_data.dropna(subset=['ConvertedComp'])
ques1_data.head()

,OpenSourcer,ConvertedComp
2,Never,8820.0
3,Never,61000.0
5,Never,366420.0
8,Once a month or more often,95179.0
9,Once a month or more often,13293.0


In [85]:
ques1_data['OpenSourcer'].value_counts()

Never                                                 19182
Less than once per year                               16547
Less than once a month but more than once per year    13673
Once a month or more often                             6421
Name: OpenSourcer, dtype: int64

In [86]:
ques1_data.groupby('OpenSourcer', as_index=False)['ConvertedComp'].mean().sort_values(by='ConvertedComp', ascending=False)

,OpenSourcer,ConvertedComp
3,Once a month or more often,135307.077714
0,Less than once a month but more than once per ...,130283.346742
1,Less than once per year,126620.355412
2,Never,122528.661453


### Question 1 answer:

As you can see from the above data, yes, OpenSource development is set to increase the developer's income. People contributing to openSource once a month or more have relitvely more income than people who don't.

### Question 2: Does your undergrad major affect youe income?

In [87]:
ques2_data = survey_data[['UndergradMajor', 'ConvertedComp']]
# Dropping all rows where either UndergradMajor or ConvertedComp is not present as Undergrad major or ConvertedComp imputing it will dilute the results
ques2_data = ques2_data.dropna()
ques2_data.head()

,UndergradMajor,ConvertedComp
2,Web development or web design,8820.0
3,"Computer science, computer engineering, or sof...",61000.0
5,Mathematics or statistics,366420.0
8,"Computer science, computer engineering, or sof...",95179.0
12,"Computer science, computer engineering, or sof...",90000.0


In [88]:
ques2_data.groupby('UndergradMajor', as_index=False)['ConvertedComp'].mean().sort_values(by='ConvertedComp', ascending=False)

,UndergradMajor,ConvertedComp
7,Fine arts or performing arts (ex. graphic desi...,180557.201559
2,"A humanities discipline (ex. literature, histo...",178508.587873
4,"A social science (ex. anthropology, psychology...",175144.201663
1,"A health science (ex. nursing, pharmacy, radio...",162801.396104
3,"A natural science (ex. biology, chemistry, phy...",153896.669127
10,Mathematics or statistics,144301.647291
8,I never declared a major,134193.229565
0,"A business discipline (ex. accounting, finance...",128246.333622
6,"Computer science, computer engineering, or sof...",126894.660743
5,"Another engineering discipline (ex. civil, ele...",119600.082778


### Question 2 answer

People with underGrad major in fine arts or performaning arts have a higher mean income. The lowest is of the web development or web design. In between, is the income of people with Mathematics of statistics major.

### Question 3: What are the languages of choice for 2020?

In [89]:
# Dropping all rows where LanguageDesireNextYear is null as it cannot be imputed
ques3_data = survey_data['LanguageDesireNextYear'].dropna()
ques3_data.head()

0    C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL
1                        C++;HTML/CSS;JavaScript;SQL
2                                    Elixir;HTML/CSS
3                                C;C#;JavaScript;SQL
4           HTML/CSS;Java;JavaScript;SQL;WebAssembly
Name: LanguageDesireNextYear, dtype: object

In [90]:
languages = {}
for row in ques3_data:
    for language in row.split(";"):
        try:
            languages[language] = languages[language] + 1
        except:
            languages[language] = 1
languages

{'C': 11143,
 'C++': 16856,
 'C#': 22449,
 'Go': 17060,
 'HTML/CSS': 36645,
 'Java': 23508,
 'JavaScript': 44739,
 'Python': 40006,
 'SQL': 33566,
 'Elixir': 3788,
 'WebAssembly': 8440,
 'Scala': 5562,
 'Bash/Shell/PowerShell': 21303,
 'Erlang': 2124,
 'Ruby': 7251,
 'Rust': 10497,
 'TypeScript': 23720,
 'Other(s):': 7912,
 'Kotlin': 13261,
 'R': 6734,
 'Assembly': 4659,
 'Objective-C': 2851,
 'PHP': 12837,
 'Swift': 8800,
 'VBA': 1641,
 'Clojure': 2752,
 'Dart': 4429,
 'F#': 3521}

In [91]:
sorted(languages)

['Assembly',
 'Bash/Shell/PowerShell',
 'C',
 'C#',
 'C++',
 'Clojure',
 'Dart',
 'Elixir',
 'Erlang',
 'F#',
 'Go',
 'HTML/CSS',
 'Java',
 'JavaScript',
 'Kotlin',
 'Objective-C',
 'Other(s):',
 'PHP',
 'Python',
 'R',
 'Ruby',
 'Rust',
 'SQL',
 'Scala',
 'Swift',
 'TypeScript',
 'VBA',
 'WebAssembly']

In [92]:
sorted_languages = sorted(languages.items(), key = lambda kv:(kv[1], kv[0]))
sorted_languages.reverse()
sorted_languages

[('JavaScript', 44739),
 ('Python', 40006),
 ('HTML/CSS', 36645),
 ('SQL', 33566),
 ('TypeScript', 23720),
 ('Java', 23508),
 ('C#', 22449),
 ('Bash/Shell/PowerShell', 21303),
 ('Go', 17060),
 ('C++', 16856),
 ('Kotlin', 13261),
 ('PHP', 12837),
 ('C', 11143),
 ('Rust', 10497),
 ('Swift', 8800),
 ('WebAssembly', 8440),
 ('Other(s):', 7912),
 ('Ruby', 7251),
 ('R', 6734),
 ('Scala', 5562),
 ('Assembly', 4659),
 ('Dart', 4429),
 ('Elixir', 3788),
 ('F#', 3521),
 ('Objective-C', 2851),
 ('Clojure', 2752),
 ('Erlang', 2124),
 ('VBA', 1641)]

### Question 3 answer:

#### Top 5 languages:

JavaScript

Python

HTML/CSS

SQL

TypeScript

#### Bottom 5 languages:

F#

Objective-C

Clojure

Erlang

VBA

### Question 4: Does Location, Open source contribution and dependents influence income?

In [93]:
survey_data.columns[survey_data.dtypes == 'float']

Index(['CompTotal', 'ConvertedComp', 'WorkWeekHrs', 'CodeRevHrs', 'Age'], dtype='object')

In [94]:
survey_data.columns[survey_data.dtypes == 'int']

Index(['Respondent'], dtype='object')

In [95]:
survey_data.columns[survey_data.dtypes == object]

Index(['MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource', 'Employment',
       'Country', 'Student', 'EdLevel', 'UndergradMajor', 'EduOther',
       'OrgSize', 'DevType', 'YearsCode', 'Age1stCode', 'YearsCodePro',
       'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney', 'MgrWant', 'JobSeek',
       'LastHireDate', 'LastInt', 'FizzBuzz', 'JobFactors', 'ResumeUpdate',
       'CurrencySymbol', 'CurrencyDesc', 'CompFreq', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'UnitTests', 'PurchaseHow', 'PurchaseWhat', 'LanguageWorkedWith',
       'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife', 'ITperson', 'OffOn',
       'SocialMedia', 'Extraversion', 'ScreenName', 'SOVi

In [96]:
def cleanData(dataFrame):
    """Clean Data
     - One of the preprocessing steps to get the data ready for training the model
        
     Keyword arguments -
     dataFrame: A pandas DataFrame that contains both numerical and categorical values
        
    """
    
    y = dataFrame['ConvertedComp']
    X = dataFrame[['Country', 'YearsCode', 'OpenSourcer', 'SOHowMuchTime', 'Dependents']]
    X['YearsCode'] = pd.to_numeric(X['YearsCode'], errors='coerce')
    X['YearsCode'].fillna(0, inplace=True)
    X['OpenSourcer'] = X['OpenSourcer'].replace({
        'Never': 0,
        'Less than once per year': 1,
        'Less than once a month but more than once per year': 2,
        'Once a month or more often': 3
    })
    X['OpenSourcer'] = pd.to_numeric(X['OpenSourcer'], downcast='integer')
    X['SOHowMuchTime'] = X['SOHowMuchTime'].replace({
        '0-10 minutes': 0,
        '11-30 minutes': 1,
        '31-60 minutes': 2,
        '60+ minutes': 3
    })
    X['SOHowMuchTime'] = pd.to_numeric(X['SOHowMuchTime'], downcast='integer')
    X['SOHowMuchTime'].fillna(0, inplace=True)
    X['Dependents'] = X['Dependents'].replace({
        'Yes': 1,
        'No': 0
    })
    X['Dependents'] = pd.to_numeric(X['Dependents'], downcast='integer')
    columns = X.columns
    num_vars = X.select_dtypes(include=['int', 'float']).columns
    for col in num_vars:
        X[col].fillna((X[col].mean()), inplace=True)
    cat_vars = X.select_dtypes(include=['object']).columns
    for var in cat_vars:
        # dropping the actual column and creating categorical columns with prefix name of original column
        X = pd.concat([X.drop([var], axis=1), pd.get_dummies(X[var], prefix=columns[columns.get_loc(var)], drop_first=True)], axis=1)
    return X, y
    
# Our predictable value is ConvertedComp so imputing it will dilute the results
X, y = cleanData(survey_data.dropna(subset=['ConvertedComp']))
final_features = X
scaleX = StandardScaler()
scaleY = StandardScaler()
X = scaleX.fit_transform(X)
y = scaleY.fit_transform(np.array(y).reshape(-1,1))

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .33, random_state=42) 

ridge = Ridge()
parameters = {
    'alpha': [
        1e-15,
        1e-10,
        1e-8,
        1e-4,
        1e-3,
        1e-2,
        1,
        5,
        10,
        20,
        50,
        100
    ]
}
ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   5, 10, 20, 50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [98]:
print(ridge_regressor.best_params_)
print("neg_mse: "+str(ridge_regressor.best_score_))

{'alpha': 5}
neg_mse: -0.905259231903392


In [99]:
final_features.columns

Index(['YearsCode', 'OpenSourcer', 'SOHowMuchTime', 'Dependents',
       'Country_Albania', 'Country_Algeria', 'Country_Andorra',
       'Country_Angola', 'Country_Argentina', 'Country_Armenia',
       ...
       'Country_United Kingdom', 'Country_United Republic of Tanzania',
       'Country_United States', 'Country_Uruguay', 'Country_Uzbekistan',
       'Country_Venezuela, Bolivarian Republic of...', 'Country_Viet Nam',
       'Country_Yemen', 'Country_Zambia', 'Country_Zimbabwe'],
      dtype='object', length=164)

In [100]:
coefficients = pd.concat([pd.DataFrame({'Feature': final_features.columns}),
                          pd.DataFrame(np.transpose(ridge_regressor.best_estimator_.coef_))], axis = 1)

In [101]:
coefficients.rename(columns={0:'Coeficient'}, inplace=True)

In [102]:
pd.set_option('display.max_rows', None)
coefficients.sort_values(by=['Coeficient'], ascending=False)

,Feature,Coeficient
156,Country_United States,2.751825e-01
154,Country_United Kingdom,8.509981e-02
10,Country_Australia,5.006614e-02
69,Country_Ireland,4.417188e-02
28,Country_Canada,4.254642e-02
0,YearsCode,4.031207e-02
139,Country_Switzerland,3.900676e-02
55,Country_Germany,3.062147e-02
106,Country_Norway,2.959913e-02
103,Country_New Zealand,2.436792e-02


### Question 4 answer: 
Yes, as you can see, the Working in:
United States, United Kingdom, Australia, Ireland and Canada lead to more income. While working in Iran, Pakistan, Russian Federaton, Brazil and India lead to a lesser developer income. Also, contributing to open source and having dependents leads to better income.

### Question 5: Does years of coding experience influence salary?

### Question 5 answer:
Yes, as observed from above, years of coding does influence te salary of a developer

### Question 6: What are the desired platforms for 2020?

In [103]:
# Dropping PlatformDesireNextYear as imputing it is not possible
platform_data = survey_data['PlatformDesireNextYear'].dropna().values
platform_data

array(['Android;Arduino;Windows', 'Windows', 'Linux;Windows', ...,
       'Linux;Raspberry Pi', 'Google Cloud Platform;Linux',
       'Android;AWS;Google Cloud Platform;IBM Cloud or Watson;iOS;Linux;MacOS;Slack;Windows'],
      dtype=object)

In [104]:
platforms = {}
for platform in platform_data:
    for i in platform.split(";"):
        try:
            temp = platforms[i]
            platforms[i] = temp+1
        except:
            platforms[i] = 1
platforms

{'Android': 24749,
 'Arduino': 11143,
 'Windows': 28215,
 'Linux': 42178,
 'Docker': 32963,
 'Kubernetes': 17793,
 'Slack': 13548,
 'Google Cloud Platform': 15916,
 'MacOS': 17433,
 'AWS': 26298,
 'IBM Cloud or Watson': 2937,
 'Raspberry Pi': 17768,
 'Heroku': 7526,
 'Other(s):': 2641,
 'WordPress': 6586,
 'iOS': 14494,
 'Microsoft Azure': 11898}

In [105]:
sorted_platforms = sorted(platforms.items(), key = lambda kv:(kv[1], kv[0]))
sorted_platforms.reverse()
sorted_platforms

[('Linux', 42178),
 ('Docker', 32963),
 ('Windows', 28215),
 ('AWS', 26298),
 ('Android', 24749),
 ('Kubernetes', 17793),
 ('Raspberry Pi', 17768),
 ('MacOS', 17433),
 ('Google Cloud Platform', 15916),
 ('iOS', 14494),
 ('Slack', 13548),
 ('Microsoft Azure', 11898),
 ('Arduino', 11143),
 ('Heroku', 7526),
 ('WordPress', 6586),
 ('IBM Cloud or Watson', 2937),
 ('Other(s):', 2641)]

### Question 6 answer:
The top 5 desired platforms of 2020 are Linux, Docker, Windows, AWS and Android

### Question 7: What are the desired databases for 2020?

In [106]:
# Dropping DatabaseDesireNextYear as imputing it is not possible
database_data = survey_data['DatabaseDesireNextYear'].dropna().values
database_data

array(['MySQL', 'MySQL', 'PostgreSQL', ..., 'PostgreSQL',
       'Firebase;MySQL;SQLite', 'MariaDB;MySQL;Oracle;SQLite'],
      dtype=object)

In [107]:
databases = {}
for database in database_data:
    for i in database.split(";"):
        try:
            temp = databases[i]
            databases[i] = temp+1
        except:
            databases[i] = 1
databases

{'MySQL': 25750,
 'PostgreSQL': 26446,
 'SQLite': 17807,
 'Couchbase': 2469,
 'Firebase': 11921,
 'MongoDB': 23667,
 'Oracle': 7299,
 'Cassandra': 6056,
 'DynamoDB': 6126,
 'Elasticsearch': 15440,
 'Redis': 17913,
 'Microsoft SQL Server': 16225,
 'MariaDB': 9986,
 'Other(s):': 4516}

In [108]:
sorted_databases = sorted(databases.items(), key = lambda kv:(kv[1], kv[0]))
sorted_databases.reverse()
sorted_databases

[('PostgreSQL', 26446),
 ('MySQL', 25750),
 ('MongoDB', 23667),
 ('Redis', 17913),
 ('SQLite', 17807),
 ('Microsoft SQL Server', 16225),
 ('Elasticsearch', 15440),
 ('Firebase', 11921),
 ('MariaDB', 9986),
 ('Oracle', 7299),
 ('DynamoDB', 6126),
 ('Cassandra', 6056),
 ('Other(s):', 4516),
 ('Couchbase', 2469)]

### Question 7 answer:
Top 5 desired databases of 2020 are PostgreSQL, MySQL, MongoDB, Redis and SQLite

### Question 8: Which is the most used editor in 2019?

In [109]:
#dropping all rows where DevEnviron column has null values as it is the column that I am trying to predict
environment_data = survey_data['DevEnviron'].dropna().values
environment_data

array(['IntelliJ;Notepad++;PyCharm', 'Atom;PyCharm',
       'Vim;Visual Studio Code', ..., 'Vim',
       'Atom;Visual Studio;Visual Studio Code',
       'Android Studio;Atom;Eclipse;NetBeans;Notepad++;Sublime Text'],
      dtype=object)

In [110]:
environments = {}
for environment in environment_data:
    for i in environment.split(";"):
        try:
            temp = environments[i]
            environments[i] = temp+1
        except:
            environments[i] = 1
environments

{'IntelliJ': 22166,
 'Notepad++': 26621,
 'PyCharm': 11724,
 'Atom': 11636,
 'Vim': 22163,
 'Visual Studio Code': 44311,
 'Eclipse': 12591,
 'Visual Studio': 27490,
 'Android Studio': 14787,
 'RStudio': 2940,
 'PHPStorm': 6645,
 'IPython / Jupyter': 8317,
 'Sublime Text': 20424,
 'Xcode': 8239,
 'NetBeans': 5121,
 'Emacs': 3922,
 'TextMate': 805,
 'Coda': 578,
 'RubyMine': 1202,
 'Zend': 307,
 'Komodo': 368,
 'Light Table': 178}

In [111]:
sorted_environments = sorted(environments.items(), key = lambda kv:(kv[1], kv[0]))
sorted_environments.reverse()
sorted_environments

[('Visual Studio Code', 44311),
 ('Visual Studio', 27490),
 ('Notepad++', 26621),
 ('IntelliJ', 22166),
 ('Vim', 22163),
 ('Sublime Text', 20424),
 ('Android Studio', 14787),
 ('Eclipse', 12591),
 ('PyCharm', 11724),
 ('Atom', 11636),
 ('IPython / Jupyter', 8317),
 ('Xcode', 8239),
 ('PHPStorm', 6645),
 ('NetBeans', 5121),
 ('Emacs', 3922),
 ('RStudio', 2940),
 ('RubyMine', 1202),
 ('TextMate', 805),
 ('Coda', 578),
 ('Komodo', 368),
 ('Zend', 307),
 ('Light Table', 178)]

### Question 8 answer:
The top 5 most used editors of 2019 are Visual Studio Code, Visual Studio, Notepad++, IntelliJ and Vim